## 1. Contagem de logins em telemóveis Android
Foi obtido um conjunto de registos a partir de logins em diferentes dispositivos. Uma amostra do dataset activity.csv é apresentada em seguida:

In [109]:
import pandas as pd

from matplotlib import pyplot as plt

df = pd.read_csv('../data/activity.csv')
df.head()

,id,user_id,created_at,device
0,1,101,2015-02-08,windows
1,2,103,2015-01-07,mac
2,3,132,2014-12-09,mac
3,4,247,2015-07-13,android
4,5,313,2015-06-20,android


In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1200 non-null   int64 
 1   user_id     1200 non-null   int64 
 2   created_at  1200 non-null   object
 3   device      1200 non-null   object
dtypes: int64(2), object(2)
memory usage: 37.6+ KB


In [111]:
df.created_at = pd.to_datetime(df.created_at)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          1200 non-null   int64         
 1   user_id     1200 non-null   int64         
 2   created_at  1200 non-null   datetime64[ns]
 3   device      1200 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 37.6+ KB


Apresente o número de entradas no dispositivo Android por mês.

In [112]:
import plotly.express as px

#plot logins mensais no device android
df_android = df[df.device == 'android']
df_android


,id,user_id,created_at,device
3,4,247,2015-07-13,android
4,5,313,2015-06-20,android
5,6,317,2014-08-08,android
8,9,443,2015-06-17,android
10,11,506,2015-04-06,android
...,...,...,...,...
1190,1191,54666,2014-08-09,android
1191,1192,54703,2015-01-29,android
1195,1196,54933,2015-04-04,android
1196,1197,54937,2014-08-05,android


In [113]:
#agregar por ano e mes
df_android['year_month'] = df_android.created_at.dt.strftime('%Y-%m')
df_android

C:\Users\TG\AppData\Local\Temp\ipykernel_18268\1566162886.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,user_id,created_at,device,year_month
3,4,247,2015-07-13,android,2015-07
4,5,313,2015-06-20,android,2015-06
5,6,317,2014-08-08,android,2014-08
8,9,443,2015-06-17,android,2015-06
10,11,506,2015-04-06,android,2015-04
...,...,...,...,...,...
1190,1191,54666,2014-08-09,android,2014-08
1191,1192,54703,2015-01-29,android,2015-01
1195,1196,54933,2015-04-04,android,2015-04
1196,1197,54937,2014-08-05,android,2014-08


In [114]:
# drop id, user_id e device
df_android = df_android.drop(['id', 'user_id', 'device'], axis=1)
df_android

,created_at,year_month
3,2015-07-13,2015-07
4,2015-06-20,2015-06
5,2014-08-08,2014-08
8,2015-06-17,2015-06
10,2015-04-06,2015-04
...,...,...
1190,2014-08-09,2014-08
1191,2015-01-29,2015-01
1195,2015-04-04,2015-04
1196,2014-08-05,2014-08


In [115]:
# contando o numero de logins por mes
df_android = df_android.groupby('year_month').count()
df_android

,created_at
year_month,
2014-07,5
2014-08,22
2014-09,27
2014-10,27
2014-11,25
2014-12,23
2015-01,14
2015-02,26
2015-03,31


In [124]:
# plotando logins mensais no device android usando plotly


px.bar(df_android, y='created_at', title='Logins mensais no device android', labels={'created_at': 'número de logins', 'year_month': 'mês'} )



## 2. Benfica vs Porto
A época desportiva 2020/21 foi extremamente competitiva e com várias reviravoltas. Dado o número de pontos do Benfica e Porto, jornada a jornada, pretende-se representar graficamente as pontuações e lideranças.

Um excerto com as pontuações do Benfica e Porto, jornada a jornada, é apresentado em seguida

In [125]:
df_times = pd.read_csv('../data/benfica_porto.csv')
df_times.head(5)

,benfica porto
0,3 3
1,6 6
2,7 9
3,10 12
4,13 15
